ZFNet implementation

In [9]:
import numpy

print(tf.__version__)
print(tf.keras.__version__)
print(numpy.__version__)

2.4.0
2.4.0
1.19.4


In [2]:
'''
ZFNet uses deconvolutional networks known as deconvnet to visualize features.
Deconvnet is the reverse of convolutional network that shows where the extracted feature comes from.
'''

import tensorflow as tf

#import mnist data
mnist = tf.keras.datasets.mnist

#divide training and test data
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

training_images = training_images[:1000]
training_labels = training_labels[:1000]
test_images = test_images[:100]
test_labels = test_labels[:100]

training_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), training_images).numpy()
test_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), test_images).numpy()

In [3]:
#resize/reshape data 
training_images = tf.image.resize(training_images, [224, 224]).numpy()
test_images = tf.image.resize(test_images, [224, 224]).numpy()

training_images = training_images.reshape(1000, 224, 224, 3)
training_images = training_images / 255.0 
test_images = test_images.reshape(100, 224, 224, 3)
test_images = test_images / 255.0

#one-hot encoding
training_labels = tf.keras.utils.to_categorical(training_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

num_len_train = int(0.8 * len(training_images))

ttraining_images = training_images[:num_len_train]
ttraining_labels = training_labels[:num_len_train]

valid_images = training_images[num_len_train:]
valid_labels = training_labels[num_len_train:]

training_images = ttraining_images
training_labels = ttraining_labels

In [4]:
#model
model = tf.keras.models.Sequential([
    #96 convolutions with 7x7 with a stride of 2, relu activation
    #3x3 max pooling with stride 2, and local contrast normalization                                
		tf.keras.layers.Conv2D(96, (7, 7), strides=(2, 2), activation='relu',
			input_shape=(224, 224, 3)),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

    #256 filters of 5x5, pooled, local contrast normalization
		tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), activation='relu'),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

    #384 filters of 3x3
		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

    #384 filters of 3x3
		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

    #256 filters of 3x3, maxpooling of 3x3 with stride 2
		tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
		tf.keras.layers.MaxPooling2D(3, strides=2),

    tf.keras.layers.Flatten(),
    #4096 neurons
		tf.keras.layers.Dense(4096),
    #4096 neurons
		tf.keras.layers.Dense(4096),
    #1000 neurons(=number of classes in ImageNet)
		tf.keras.layers.Dense(10, activation='softmax')
	])

'''
Local Contrast Normalization is a type of normalization that performs local subtraction and division normalizations, 
enforcing a sort of local competition between adjacent features in a feature map, 
and between features at the same spatial location in different feature maps.
'''

#compile model
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9), 
              loss='categorical_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(5)])

'''
TopK Categorical Accuracy calculates the percentage of records 
for which the targets are in the top K predictions.
'''

#callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                            		factor=0.1, patience=1, 
																								min_lr=0.00001)

#train model
model.fit(training_images, training_labels, batch_size=128, 
          validation_data=(valid_images, valid_labels), 
					epochs=50, callbacks=[reduce_lr])

Epoch 1/50
7/7 [==============================] - 91s 13s/step - loss: 2.6557 - accuracy: 0.1232 - top_k_categorical_accuracy: 0.5284 - val_loss: 2.6267 - val_accuracy: 0.2250 - val_top_k_categorical_accuracy: 0.5650
Epoch 2/50
7/7 [==============================] - 87s 12s/step - loss: 2.1445 - accuracy: 0.2890 - top_k_categorical_accuracy: 0.7308 - val_loss: 1.5845 - val_accuracy: 0.4700 - val_top_k_categorical_accuracy: 0.8600
Epoch 3/50
7/7 [==============================] - 87s 12s/step - loss: 1.8709 - accuracy: 0.4746 - top_k_categorical_accuracy: 0.8248 - val_loss: 1.7319 - val_accuracy: 0.4850 - val_top_k_categorical_accuracy: 0.8400
Epoch 4/50
7/7 [==============================] - 87s 12s/step - loss: 1.7710 - accuracy: 0.5717 - top_k_categorical_accuracy: 0.8787 - val_loss: 1.6732 - val_accuracy: 0.5750 - val_top_k_categorical_accuracy: 0.9200
Epoch 5/50
7/7 [==============================] - 87s 12s/step - loss: 1.8393 - accuracy: 0.5353 - top_k_categorical_accuracy: 0.932

In [6]:
#Evaluate model
print("\n Accuracy: %.4f" % (model.evaluate(test_images, test_labels)[1]))

4/4 [==============================] - 3s 643ms/step - loss: 0.6950 - accuracy: 0.8300 - top_k_categorical_accuracy: 0.9900

 Accuracy: 0.8300
